In [ ]:
# | default_exp airflow.executor

In [ ]:
# | export

from os import environ
from typing import *

import airt_service.sanitizer
from airt.executor.subcommand import CLICommandBase
from airt.logger import get_logger
from airt_service.airflow.aws_batch_executor import AirflowAWSBatchExecutor
from airt_service.airflow.azure_batch_executor import AirflowAzureBatchExecutor
from airt_service.airflow.base_executor import BaseAirflowExecutor
from airt_service.airflow.bash_executor import AirflowBashExecutor

22-12-19 09:00:02.575 [INFO] airt.executor.subcommand: Module loaded.


In [ ]:
import tempfile
from pathlib import Path

import yaml
from airt.executor.subcommand import ClassCLICommand, SimpleCLICommand
from airt.testing import activate_by_import
from airt_service.airflow.utils import wait_for_run_to_complete
from airt_service.aws.batch_utils import (
    create_default_batch_environment_config,
    create_testing_batch_environment_ctx,
)
from airt_service.helpers import generate_random_string, set_env_variable_context

[INFO] airt.testing.activate_by_import: Testing environment activated.


2022-12-19 09:00:03.152396: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[WARNING] airt.testing.activate_by_import: Failed to set gpu memory limit for tf; This could happen because of no gpu availability


In [ ]:
# | exporti

logger = get_logger(__name__)

In [ ]:
logger.info("Module loaded.")

[INFO] __main__: Module loaded.


In [ ]:
def setup_test_paths(d: str) -> Tuple[str, str]:
    d = Path(d)
    paths = [d / sd for sd in ["data", "model"]]
    display(f"{paths=}")

    # create tmp dirs for data and model
    for p in paths:
        p.mkdir(parents=True, exist_ok=True)

    # RemotePaths: data_path is "read-only", while model_path can be used for both reading and writing between calls
    return tuple(f"local:{p}" for p in paths)


with tempfile.TemporaryDirectory() as d:
    data_path_url, model_path_url = setup_test_paths(d)

data_path_url, model_path_url

"paths=[Path('/tmp/tmpzozxgbyu/data'), Path('/tmp/tmpzozxgbyu/model')]"

('local:/tmp/tmpzozxgbyu/data', 'local:/tmp/tmpzozxgbyu/model')

In [ ]:
# | export


class AirflowExecutor:
    @classmethod
    def create_executor(
        cls, steps: List[CLICommandBase], cloud_provider: str, **kwargs
    ) -> BaseAirflowExecutor:
        """
        Initialize and return airflow bash or batch executor based on env variable

        Args:
            steps: list of instances of either ClassCLICommand or SimpleCLICommand
            cloud_provider: cloud provider to executor batch job
            kwargs: additional keyword arguments to pass to init
        Returns:
            An instance of AirflowBashExecutor or AirflowAWSBatchExecutor
        """

        job_executor = environ.get("JOB_EXECUTOR", None)

        if job_executor == "aws":
            if cloud_provider == "azure":
                executor = AirflowAzureBatchExecutor(steps=steps, **kwargs)
            else:
                executor = AirflowAWSBatchExecutor(steps=steps, **kwargs)
        else:
            executor = AirflowBashExecutor(steps=steps)

        return executor

In [ ]:
steps = [
    ClassCLICommand(
        executor_name="test-executor", class_name="MyTestExecutor", f_name="f"
    ),
    ClassCLICommand(
        executor_name="test-executor", class_name="MyTestExecutor", f_name="g"
    ),
]

on_step_start = SimpleCLICommand(command="sleep {step_count}")
on_step_end = SimpleCLICommand(command="echo step {step_count} completed")

In [ ]:
def save_test_batch_environment_arns(folder: Path):
    test_batch_environment_arns = {
        "eu-west-1": {
            task: {
                arn: "arn:aws:batch:placeholder"
                for arn in [
                    "compute_environment_arn",
                    "job_definition_arn",
                    "job_queue_arn",
                ]
            }
            for task in ["csv_processing", "predictions", "preprocessing", "training"]
        }
    }

    folder = Path(folder)
    test_batch_environment_arn_path = folder / "batch_environment.yml"
    with open(test_batch_environment_arn_path, "w") as f:
        yaml.dump(test_batch_environment_arns, f, default_flow_style=False)

    return test_batch_environment_arn_path

In [ ]:
executor = AirflowExecutor.create_executor(steps, "aws")
display(type(executor))
assert isinstance(executor, AirflowBashExecutor)

with set_env_variable_context(variable="JOB_EXECUTOR", value="fastapi"):
    executor = AirflowExecutor.create_executor(steps, "aws")
    display(type(executor))
    assert isinstance(executor, AirflowBashExecutor)

with tempfile.TemporaryDirectory() as d:
    test_batch_environment_arn_path = save_test_batch_environment_arns(d)
    with set_env_variable_context(variable="JOB_EXECUTOR", value="aws"):
        executor = AirflowExecutor.create_executor(
            steps,
            cloud_provider="aws",
            region="eu-west-1",
            exec_environments=["training", "training"],
            batch_environment_arn_path=test_batch_environment_arn_path,
        )
        display(type(executor))
        assert isinstance(executor, AirflowAWSBatchExecutor)

airt_service.airflow.bash_executor.AirflowBashExecutor

airt_service.airflow.bash_executor.AirflowBashExecutor

airt_service.airflow.aws_batch_executor.AirflowAWSBatchExecutor

In [ ]:
with tempfile.TemporaryDirectory() as d:
    data_path_url, model_path_url = setup_test_paths(d)

    with set_env_variable_context(variable="JOB_EXECUTOR", value="fastapi"):
        executor = AirflowExecutor.create_executor(steps, cloud_provider="aws")
        display(type(executor))
        assert isinstance(executor, AirflowBashExecutor)

        dag_file_path, run_id = executor.execute(
            description="test description",
            tags="test_tag",
            on_step_start=on_step_start,
            on_step_end=on_step_end,
            data_path_url=data_path_url,
            model_path_url=model_path_url,
        )
        display(dag_file_path)
        display(run_id)

        dag_id = str(dag_file_path).split("/")[-1].split(".py")[0]
        state = wait_for_run_to_complete(dag_id=dag_id, run_id=run_id, timeout=600)
        display(state)
        dag_file_path.unlink()

"paths=[Path('/tmp/tmpsyqro8dt/data'), Path('/tmp/tmpsyqro8dt/model')]"

airt_service.airflow.bash_executor.AirflowBashExecutor

CompletedProcess(args=['/home/kumaran/airflow_venv/bin/airflow', 'dags', 'trigger', 'test-executor-my_test_executor-f-data-path-urllocaltmptmpsyqro8dtdata-model-path-urllocaltmptmpsyqro8dtmodel_test-executor-my_test_executor-g-data-path-urllocaltmptmpsyqro8dtdata-model-path-urllocaltmptmpsyqro8dtmodel', '--conf', '{"data_path_url": "local:/tmp/tmpsyqro8dt/data", "model_path_url": "local:/tmp/tmpsyqro8dt/model"}', '--run-id', 'airt-service__2022-12-19T09:00:12.953139'], returncode=0, stdout='[\x1b2022-12-19 09:00:14,013\x1b[0m] {\x1b__init__.py:\x1b42} INFO\x1b - Loaded API auth backend: airflow.api.auth.backend.session\x1b\nCreated <DagRun test-executor-my_test_executor-f-data-path-urllocaltmptmpsyqro8dtdata-model-path-urllocaltmptmpsyqro8dtmodel_test-executor-my_test_executor-g-data-path-urllocaltmptmpsyqro8dtdata-model-path-urllocaltmptmpsyqro8dtmodel @ 2022-12-19T09:00:14+00:00: airt-service__2022-12-19T09:00:12.953139, state:queued, queued_at: 2022-12-19 09:00:14.128843+00:00. exte

Path('/home/kumaran/airflow/dags/test-executor-my_test_executor-f-data-path-urllocaltmptmpsyqro8dtdata-model-path-urllocaltmptmpsyqro8dtmodel_test-executor-my_test_executor-g-data-path-urllocaltmptmpsyqro8dtdata-model-path-urllocaltmptmpsyqro8dtmodel.py')

'airt-service__2022-12-19T09:00:12.953139'

'success'

In [ ]:
# | eval: false

region = "eu-west-1"

with tempfile.TemporaryDirectory() as d:
    data_path_url, model_path_url = setup_test_paths(d)

    exec_environments = ["training", "predictions"]

    td = Path(d)
    env_config_path = td / "env_config.yaml"
    created_env_info_path = td / "output_file.yaml"
    create_default_batch_environment_config(
        prefix=f"airflow_batch_execute_testing_{generate_random_string()}",
        output_path=env_config_path,
        regions=[region],
    )

    with open(env_config_path) as f:
        env_config = yaml.safe_load(f)
    display(f"{env_config=}")
    with create_testing_batch_environment_ctx(
        input_yaml_path=env_config_path, output_yaml_path=created_env_info_path
    ):
        with set_env_variable_context(variable="JOB_EXECUTOR", value="aws"):
            executor = AirflowExecutor.create_executor(
                steps=steps,
                cloud_provider="aws",
                region=region,
                exec_environments=exec_environments,
                batch_environment_arn_path=created_env_info_path,
            )

            dag_file_path, run_id = executor.execute(
                description="test description",
                tags="test_tag",
                on_step_start=on_step_start,
                on_step_end=on_step_end,
                data_path_url=data_path_url,
                model_path_url=model_path_url,
            )
            display(dag_file_path)
            display(run_id)

            dag_id = str(dag_file_path).split("/")[-1].split(".py")[0]
            state = wait_for_run_to_complete(dag_id=dag_id, run_id=run_id, timeout=3600)
            display(state)
            dag_file_path.unlink()

"paths=[Path('/tmp/tmpkry2gw4o/data'), Path('/tmp/tmpkry2gw4o/model')]"

"env_config={'eu-west-1': {'csv_processing': {'compute_environment': {'instance_type': 'r5.16xlarge', 'max_instances': 10, 'min_instances': 0, 'name': 'airflow_batch_execute_testing_1785JN_csv_processing_compute_environment'}, 'job_definition': {'image': 'ghcr.io/airtai/airt-service:dev', 'name': 'airflow_batch_execute_testing_1785JN_csv_processing_job_definition'}, 'job_queue': {'name': 'airflow_batch_execute_testing_1785JN_csv_processing_job_queue', 'priority': 100}}, 'predictions': {'compute_environment': {'instance_type': 'g4dn.xlarge', 'max_instances': 10, 'min_instances': 0, 'name': 'airflow_batch_execute_testing_1785JN_predictions_compute_environment'}, 'job_definition': {'image': 'ghcr.io/airtai/airt-service:dev', 'name': 'airflow_batch_execute_testing_1785JN_predictions_job_definition'}, 'job_queue': {'name': 'airflow_batch_execute_testing_1785JN_predictions_job_queue', 'priority': 100}}, 'preprocessing': {'compute_environment': {'instance_type': 'r5.16xlarge', 'max_instances'

task_name='csv_processing'
[INFO] botocore.credentials: Found credentials in environment variables.
[INFO] airt_service.aws.batch_utils: wait(): self.arn='arn:aws:batch:eu-west-1:617504802562:compute-environment/airflow_batch_execute_testing_1785JN_csv_processing_compute_environment', status=CREATING, state=ENABLED
[INFO] airt_service.aws.batch_utils: wait(): self.arn='arn:aws:batch:eu-west-1:617504802562:compute-environment/airflow_batch_execute_testing_1785JN_csv_processing_compute_environment', status=VALID, state=ENABLED
compute_env.arn='arn:aws:batch:eu-west-1:617504802562:compute-environment/airflow_batch_execute_testing_1785JN_csv_processing_compute_environment'
[INFO] airt_service.aws.batch_utils: wait(): self.arn='arn:aws:batch:eu-west-1:617504802562:job-queue/airflow_batch_execute_testing_1785JN_csv_processing_job_queue', status=CREATING, state=ENABLED
[INFO] airt_service.aws.batch_utils: wait(): self.arn='arn:aws:batch:eu-west-1:617504802562:job-queue/airflow_batch_execute_t

Path('/home/kumaran/airflow/dags/test-executor-my_test_executor-f-data-path-urllocaltmptmpkry2gw4odata-model-path-urllocaltmptmpkry2gw4omodel_test-executor-my_test_executor-g-data-path-urllocaltmptmpkry2gw4odata-model-path-urllocaltmptmpkry2gw4omodel.py')

'airt-service__2022-12-19T09:01:45.116698'

...

[INFO] airt_service.aws.batch_utils: wait(): self.arn='arn:aws:batch:eu-west-1:617504802562:job-queue/airflow_batch_execute_testing_1785JN_csv_processing_job_queue', status=DELETING, state=DISABLED
[INFO] airt_service.aws.batch_utils: wait(): self.arn='arn:aws:batch:eu-west-1:617504802562:job-queue/airflow_batch_execute_testing_1785JN_csv_processing_job_queue', status=DELETING, state=DISABLED
[INFO] airt_service.aws.batch_utils: wait(): self.arn='arn:aws:batch:eu-west-1:617504802562:job-queue/airflow_batch_execute_testing_1785JN_csv_processing_job_queue' deleted
deleting compute env - csv_processing
[INFO] airt_service.aws.batch_utils: wait(): self.arn='arn:aws:batch:eu-west-1:617504802562:compute-environment/airflow_batch_execute_testing_1785JN_csv_processing_compute_environment', status=VALID, state=DISABLED
deleting job definition - predictions
deleting job queue - predictions
[INFO] airt_service.aws.batch_utils: wait(): self.arn='arn:aws:batch:eu-west-1:617504802562:job-queue/airfl

[INFO] airt_service.aws.batch_utils: wait(): self.arn='arn:aws:batch:eu-west-1:617504802562:job-queue/airflow_batch_execute_testing_1785JN_predictions_job_queue', status=DELETING, state=DISABLED
[INFO] airt_service.aws.batch_utils: wait(): self.arn='arn:aws:batch:eu-west-1:617504802562:job-queue/airflow_batch_execute_testing_1785JN_predictions_job_queue' deleted
deleting compute env - predictions
[INFO] airt_service.aws.batch_utils: wait(): self.arn='arn:aws:batch:eu-west-1:617504802562:compute-environment/airflow_batch_execute_testing_1785JN_predictions_compute_environment', status=UPDATING, state=DISABLED
[INFO] airt_service.aws.batch_utils: wait(): self.arn='arn:aws:batch:eu-west-1:617504802562:compute-environment/airflow_batch_execute_testing_1785JN_predictions_compute_environment', status=UPDATING, state=DISABLED
[INFO] airt_service.aws.batch_utils: wait(): self.arn='arn:aws:batch:eu-west-1:617504802562:compute-environment/airflow_batch_execute_testing_1785JN_predictions_compute_e

[INFO] airt_service.aws.batch_utils: wait(): self.arn='arn:aws:batch:eu-west-1:617504802562:job-queue/airflow_batch_execute_testing_1785JN_preprocessing_job_queue', status=DELETING, state=DISABLED
[INFO] airt_service.aws.batch_utils: wait(): self.arn='arn:aws:batch:eu-west-1:617504802562:job-queue/airflow_batch_execute_testing_1785JN_preprocessing_job_queue', status=DELETING, state=DISABLED
[INFO] airt_service.aws.batch_utils: wait(): self.arn='arn:aws:batch:eu-west-1:617504802562:job-queue/airflow_batch_execute_testing_1785JN_preprocessing_job_queue', status=DELETING, state=DISABLED
[INFO] airt_service.aws.batch_utils: wait(): self.arn='arn:aws:batch:eu-west-1:617504802562:job-queue/airflow_batch_execute_testing_1785JN_preprocessing_job_queue', status=DELETING, state=DISABLED
[INFO] airt_service.aws.batch_utils: wait(): self.arn='arn:aws:batch:eu-west-1:617504802562:job-queue/airflow_batch_execute_testing_1785JN_preprocessing_job_queue', status=DELETING, state=DISABLED
[INFO] airt_ser

[INFO] airt_service.aws.batch_utils: wait(): self.arn='arn:aws:batch:eu-west-1:617504802562:job-queue/airflow_batch_execute_testing_1785JN_training_job_queue', status=DELETING, state=DISABLED
[INFO] airt_service.aws.batch_utils: wait(): self.arn='arn:aws:batch:eu-west-1:617504802562:job-queue/airflow_batch_execute_testing_1785JN_training_job_queue', status=DELETING, state=DISABLED
[INFO] airt_service.aws.batch_utils: wait(): self.arn='arn:aws:batch:eu-west-1:617504802562:job-queue/airflow_batch_execute_testing_1785JN_training_job_queue', status=DELETING, state=DISABLED
[INFO] airt_service.aws.batch_utils: wait(): self.arn='arn:aws:batch:eu-west-1:617504802562:job-queue/airflow_batch_execute_testing_1785JN_training_job_queue', status=DELETING, state=DISABLED
[INFO] airt_service.aws.batch_utils: wait(): self.arn='arn:aws:batch:eu-west-1:617504802562:job-queue/airflow_batch_execute_testing_1785JN_training_job_queue', status=DELETING, state=DISABLED
[INFO] airt_service.aws.batch_utils: wai

[INFO] airt_service.aws.batch_utils: wait(): self.arn='arn:aws:batch:eu-west-1:617504802562:compute-environment/airflow_batch_execute_testing_1785JN_training_compute_environment', status=UPDATING, state=DISABLED
[INFO] airt_service.aws.batch_utils: wait(): self.arn='arn:aws:batch:eu-west-1:617504802562:compute-environment/airflow_batch_execute_testing_1785JN_training_compute_environment', status=UPDATING, state=DISABLED
[INFO] airt_service.aws.batch_utils: wait(): self.arn='arn:aws:batch:eu-west-1:617504802562:compute-environment/airflow_batch_execute_testing_1785JN_training_compute_environment', status=UPDATING, state=DISABLED
[INFO] airt_service.aws.batch_utils: wait(): self.arn='arn:aws:batch:eu-west-1:617504802562:compute-environment/airflow_batch_execute_testing_1785JN_training_compute_environment', status=UPDATING, state=DISABLED
[INFO] airt_service.aws.batch_utils: wait(): self.arn='arn:aws:batch:eu-west-1:617504802562:compute-environment/airflow_batch_execute_testing_1785JN_tra